In [1]:
import pandas as pd
import numpy as np

def process_metrics(metric_data):
    # Create a DataFrame from the list of strings
    data = {
        "Metric": [],
        "Training": [],
        "Validation": []
    }
    
    for item in metric_data:
        parts = item.split()  # Split string into components
        metric_name = " ".join(parts[:-2])  # Metric name is all except last two elements
        training_val = parts[-2]  # Second last element is training value
        validation_val = parts[-1]  # Last element is validation value
        
        # Convert 'NaN' string to actual NaN value
        training_val = float(training_val) if training_val != "NaN" else np.nan
        validation_val = float(validation_val) if validation_val != "NaN" else np.nan
        
        data['Metric'].append(metric_name)
        data['Training'].append(training_val)
        data['Validation'].append(validation_val)

    df = pd.DataFrame(data)
    
    # Calculate averages and standard deviations
    summary_data = {
        'Statistic': ['Average', 'Standard Deviation'],
        'Average Training': [df['Training'].mean(), df['Training'].std()],
        'Average Validation': [df['Validation'].mean(), df['Validation'].std()]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Formatting output
    result = "Detailed Metrics:\n" + df.to_string(index=False) + "\n\nSummary Statistics:\n" + summary_df.to_string(index=False)
    
    return result

# Example usage
metric_lines = [
    "Average Accuracy 0.661 0.611",
    "Standard Deviation of Accuracy 0.250 0.169",
    "Best Accuracy 0.946 0.798",
    "Last Accuracy 0.935 0.798",
    "Overall AUC NaN 0.540",
    "Maximum F1 Score NaN 0.804",
    "Minimum Loss NaN 0.738",
    "Difference in Average Loss Last N NaN 0.655",
    "Standard Deviation of Loss Last N NaN 0.104"
]

result = process_metrics(metric_lines)
print(result)


Detailed Metrics:
                           Metric  Training  Validation
                 Average Accuracy     0.661       0.611
   Standard Deviation of Accuracy     0.250       0.169
                    Best Accuracy     0.946       0.798
                    Last Accuracy     0.935       0.798
                      Overall AUC       NaN       0.540
                 Maximum F1 Score       NaN       0.804
                     Minimum Loss       NaN       0.738
Difference in Average Loss Last N       NaN       0.655
Standard Deviation of Loss Last N       NaN       0.104

Summary Statistics:
         Statistic  Average Training  Average Validation
           Average          0.698000            0.579667
Standard Deviation          0.326469            0.267832
